# GNSS time series decomposition

Gareth Funning, University of California, Riverside

Let's have a look at some processed GNSS data. We'll look at a couple of things $-$ where the stations are, and after selecting one station, we'll grab the time series for it and try to detrend it.

[Note that this is a demonstration notebook testing some of the elements of a longer exercise. In theory, if these parts work then the longer thing should, too...]

## 0. Dependencies and tokens

First: dependencies. If these aren't already installed, Jupyter will be sad.

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pygmt
import requests
from earthscope_sdk.auth.device_code_flow import DeviceCodeFlowSimple
from earthscope_sdk.auth.auth_flow import NoTokensError

Second, define your EarthScope download credentials by pointing to the location of your Single Sign-On (SSO) token.

In [ ]:
# set up your EarthScope credentials here

# path to the directory where you keep your tokens
token_path = "/path/to/your/token/directory"  # THIS IS A FAKE NAME!
token_path = "/home/gareth/pkgs/tokens"
# example: "/home/gareth/pkgs/tokens"

# instantiate the device code flow subclass
device_flow = DeviceCodeFlowSimple(Path(token_path))
try:
    # get access token from local path
    device_flow.get_access_token_refresh_if_necessary()
except NoTokensError:
    # if no token was found locally, do the device code flow
    device_flow.do_flow()
token = device_flow.access_token

## 1. Let's look for some stations

Let's look for GNSS stations in the vicinity of the $M_w$7.1 El Mayor-Cucapah earthquake that occurred near the city of Mexicali, just south of the Mexican border with California on April 4th, 2010. The GNSS time series for stations that were established before that earthquake, which included many of the Plate Boundary Observatory (PBO; now known as the Network of the Americas, or NOTA) stations in the region, have seen a range of processes $-$ strain accumulation due to relative plate motion, the earthquake, the ensuing postseismic deformation transient, as well as seasonal deformation and who knows what else!

For a first task, we can download a file containing station locations, and plot them. On the basis of that, we can choose a station to look at in more detail.

EarthScope is the source of many useful things GNSS, including the archive of RINEX data for the NOTA, as well as its processed time series and velocities. These come under the category 'GNSS Products' on their website. I propose downloading a file of processed velocities, which includes the station locations. [EarthScope has just the sorts of things we need](https://gage-data.earthscope.org/archive/gnss/products/velocity/)!

In [ ]:
# want to see where continuous GNSS stations are? 
# let's download a file containing the processed velocities!
# note that EarthScope's GNSS data live here: https://gage-data.earthscope.org/archive/gnss

# location of a file with processed GNSS velocities:
url = "https://gage-data.earthscope.org/archive/gnss/products/velocity/cwu.final_nam14.vel"

# this code block does the data download using your credentials

# request a file and provide the token in the Authorization header
file_name = Path(url).name
directory_to_save_file = Path.cwd() # where you want to save the downloaded file 

r = requests.get(url, headers={"authorization": f"Bearer {token}"})
if r.status_code == requests.codes.ok:
    # save the file
    with open(Path(directory_to_save_file /
 file_name), 'wb') as f:
        for data in r:
            f.write(data)
else:
    #problem occured
    print(f"failure: {r.status_code}, {r.reason}")

I'm not going to dwell on all of the detailed information that is contained in this file right now, but if you are interested, you could have a look at it in a text editor. Right now I am going to gloss over what it contains, and use a couple of shell commands to extract station locations and names out of it, which we can read in to Python and then plot on a map...

In [ ]:
# use a terminal command to extract the parts of the file that we want! (long, lat and station name)
!awk '{if (NR>36) print $9, $8, $1}' cwu.final_nam14.vel > tmp
!awk 'BEGIN{site="1LSU"}{if ($NF!=site) print line; site=$NF; line=$0}END{print line}' tmp > cwu.final_nam14.txt

# and let's read them in as a pandas dataframe!
nota_stas=pd.read_csv('cwu.final_nam14.txt', sep=' ', header=None)
nota_stas.columns=['lon','lat','name']

In [ ]:
# if you want to see what we just loaded in, run this cell
nota_stas

If we have successfully loaded in some data, we can plot the station locations on a map. PyGMT is good for this sort of thing. (Everybody should learn how to plot things in GMT!)

In [ ]:
# and now let's plot the station locations

# select a bounding box for the map
bbox = [-117.5, -114.5, 32.0, 33.5] # [west, east, south, north]

# initiate a PyGMT figure 
fig = pygmt.Figure() 

# override some ugly (I think) defaults:
pygmt.config(FORMAT_GEO_MAP="ddd.x")

# make a basemap frame
fig.basemap(region=bbox, projection="M22c", frame=["a1f0.2","WeSn"]) 

# plot coasts in navy blue and national borders in dark red
fig.coast(shorelines=["1/0.5p,navy","2/0.5p,navy"], borders=["1/0.5p,darkred","2/0.5p,darkred,-"]) 

# i took the liberty of looking up the USGS earthquake epicenter: plot it as a gold star
fig.plot(x=-115.295,y=32.286,style="a1c",fill="gold",pen="0.5p,black")

# plot all of the stations as red inverted triangles
fig.plot(x=nota_stas['lon'],y=nota_stas['lat'],style="i0.4c",fill="red",pen="0.5p,black")

# and label them
fig.text(x=nota_stas['lon'],y=nota_stas['lat'], text=nota_stas['name'],justify='TL',font='8p')

# and display!
fig.show(width=800)
#fig.savefig('nota_map.png')

## 2. Time series futzing

A prelude to a longer exercise. Let's pick a GNSS station, download the time series for it, and do some very simple analysis.

Look at the map: there are a fair number of stations within 100 km (approximately 1 degree of latitude) of the epicenter. In fact, the earthquake generated a surface rupture over 100 km, both to the NW and SE of the epicenter; the northwestern tip of the rupture crossed the border into the US. (Javier Gonzalez Garcia: "Earthquakes don't respect international borders!") A lot of these stations will have recorded significant displacements for the earthquake. 

The stations north of the border with codes Pxxx (where xxx is a 3-digit number) are PBO stations; of these only P796 was installed after the earthquake, the others were installed several years before it. Let's pick one of them and have a look at the data.

In [ ]:
# pick yer GNSS station here:
my_sta='P501'   # enter a four character code between the quotes

# and assemble the url of the file we want to download (and its informational webpage)
datafile="{0:s}.cwu.nam14.csv".format(my_sta)
dirurl="https://gage-data.earthscope.org/archive/gnss/products/position/{0:s}/".format(my_sta)
dataurl=dirurl+datafile
siteurl="https://www.unavco.org/instrumentation/networks/status/nota/overview/{0:s}".format(my_sta)

# print some information 
print("Datafile is {0:s}".format(datafile))
print("Directory URL is {0:s}".format(dirurl))
print("Data URL is {0:s}".format(dataurl))
print("Station webpage is {0:s}".format(siteurl))  # go and have a look if you're curious 

Assuming the URLs look right, the next step is downloading the time series file:

In [ ]:
# request a file and provide the token in the Authorization header
file_name = Path(dataurl).name
directory_to_save_file = Path.cwd() # where you want to save the downloaded file 

r = requests.get(dataurl, headers={"authorization": f"Bearer {token}"})
if r.status_code == requests.codes.ok:
    # save the file
    with open(Path(directory_to_save_file /
 file_name), 'wb') as f:
        for data in r:
            f.write(data)
else:
    #problem occured
    print(f"failure: {r.status_code}, {r.reason}")

We can try and use a shell command to see what we downloaded:

In [ ]:
# use a shell command to view the file with line numbers
# (useful to figure out how many lines to skip)
#!cat -n $datafile
!cat -n P501.cwu.nam14.csv

...and on the basis of the file contents (how many header lines, how many columns), ingest it into a pandas dataframe.

In [ ]:
# skip 11 header lines, there are 8 columns, ignore spaces in lines, using the python engine to parse lines
gnss_data=pd.read_csv(datafile,header=11, usecols=np.arange(0,8), sep=r'\s*,\s*', engine='python')

# and tell it that the first column is made up of dates (it doesn't seems to be able to figure that out on its own)
gnss_data['Date'] = pd.to_datetime(gnss_data['Date'])

# add a column for the day number
gnss_data['Day_number'] = (gnss_data['Date'] - gnss_data['Date'].min())  / np.timedelta64(1,'D')

You can view the contents if you like:

In [ ]:
#gnss_data["East (mm)"][0:20]
#gnss_data.iloc[5]
gnss_data

And we can plot the displacements to see what they look like, using matplotlib:

In [ ]:
# let's see what we've got here...

# date formatting, copied from the docs
years_fmt = mdates.DateFormatter('%Y')

fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, figsize=(15,15))
fig.suptitle(my_sta)

ax1.plot_date(gnss_data['Date'],gnss_data['East (mm)'],fmt='.')
ax1.set_ylabel('East (mm)')
ax1.xaxis.set_major_formatter(years_fmt)

ax2.plot_date(gnss_data['Date'],gnss_data['North (mm)'],fmt='.')
ax2.set_ylabel('North (mm)')
ax2.xaxis.set_major_formatter(years_fmt)

ax3.plot_date(gnss_data['Date'],gnss_data['Vertical (mm)'],fmt='.')
ax3.set_ylabel('Vertical (mm)')
ax3.xaxis.set_major_formatter(years_fmt)

fig.autofmt_xdate()

fig.show()

I noticed that a couple of the stations had some pretty large outliers early on, which (somewhat encouragingly) also had some very large errors associated with them. We can (optionally) exclude the points with the largest uncertainties if you want...

In [ ]:
# suppress outliers?

# the standard deviation of most data points' horizontal components is about 2 mm
# data points with sigmas worse than 8 mm are probably very bad!
# run this and re-run the plot and see if it looks any better
gnss_data=gnss_data.loc[(gnss_data[['North Std. Deviation (mm)','East Std. Deviation (mm)']] < 8).all(axis=1)]

...and you should go back and re-run the plotting cell and see if it made anything better.

### 2.1 Detrending the data

The next step will be an attempt to estimate the pre-earthquake deformation rate, and remove it from the data, by selecting the data from before the earthquake and fitting a straight line to it via a linear inversion.

In [ ]:
# detrending the data

# when did the earthquake happen?
eq_date=np.datetime64('2010-04-04')

# select a subset of the data before the El Mayor-Cucapah earthquake
pre_eq=gnss_data.loc[gnss_data['Date']<eq_date]

# make some normal equations
AT = np.vstack((pre_eq['Day_number'], np.ones(np.shape(pre_eq['Date']))))

# east component
Einv_e = np.diag(1/pre_eq['East Std. Deviation (mm)']**2)
ATAinv_e = np.linalg.inv(np.matmul(AT,np.matmul(Einv_e,AT.T)))
ATd_e = np.matmul(AT,np.matmul(Einv_e,pre_eq['East (mm)']))
m_e = np.matmul(ATAinv_e,ATd_e.T)  # slope and intercept of best-fit line

# north component
Einv_n = np.diag(1/pre_eq['North Std. Deviation (mm)']**2)
ATAinv_n = np.linalg.inv(np.matmul(AT,np.matmul(Einv_n,AT.T)))
ATd_n = np.matmul(AT,np.matmul(Einv_n,pre_eq['North (mm)']))
m_n = np.matmul(ATAinv_n,ATd_n.T)  # slope and intercept of best-fit line

# vertical component
Einv_v = np.diag(1/pre_eq['Vertical Std. Deviation (mm)']**2)
ATAinv_v = np.linalg.inv(np.matmul(AT,np.matmul(Einv_v,AT.T)))
ATd_v = np.matmul(AT,np.matmul(Einv_v,pre_eq['Vertical (mm)']))
m_v = np.matmul(ATAinv_v,ATd_v.T)  # slope and intercept of best-fit line

# and let them know the answer!
print('Best-fit pre-event velocity:')
# remember our estimated velocities are per day, so we'll want to scale them up a bit
print('  x = {0:6.2f} +/- {1:4.2f} mm/yr'.format(m_e[0]*365.25,np.sqrt(ATAinv_e[0][0])*365.25))
print('  y = {0:6.2f} +/- {1:4.2f} mm/yr'.format(m_n[0]*365.25,np.sqrt(ATAinv_n[0][0])*365.25))
print('  z = {0:6.2f} +/- {1:4.2f} mm/yr'.format(m_v[0]*365.25,np.sqrt(ATAinv_v[0][0])*365.25))

Well, that was fun! To conclude, let's see what the time series look like with those pre-earthquake velocities removed:

In [ ]:
# evaluate the straight lines!
e_trend=gnss_data['Day_number']*m_e[0]+m_e[1]
n_trend=gnss_data['Day_number']*m_n[0]+m_n[1]
v_trend=gnss_data['Day_number']*m_v[0]+m_v[1]

# as a check, let's detrend our data using this pre-earthquake velocity!

fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, figsize=(15,15))
fig.suptitle('{0:s} detrended'.format(my_sta))

ax1.plot_date(gnss_data['Date'],gnss_data['East (mm)']-e_trend,fmt='.')
ax1.set_ylabel('East (mm)')
ax1.xaxis.set_major_formatter(years_fmt)

ax2.plot_date(gnss_data['Date'],gnss_data['North (mm)']-n_trend,fmt='.')
ax2.set_ylabel('North (mm)')
ax2.xaxis.set_major_formatter(years_fmt)

ax3.plot_date(gnss_data['Date'],gnss_data['Vertical (mm)']-v_trend,fmt='.')
ax3.set_ylabel('Vertical (mm)')
ax3.xaxis.set_major_formatter(years_fmt)

fig.autofmt_xdate()

fig.show()

Hopefully they look flatter than they did before?

### 2.2 Correcting offsets

Next up: an offset correction. The earthquake caused an offset in the position time series. We can try and estimate this by differencing pre- and post-event positions 

In [ ]:
# look at the data either side of the offset
gnss_data.loc[((gnss_data['Date']>eq_date-5)&(gnss_data['Date']<eq_date+5))]

In [ ]:
# we will make some arrays to store any offset correction(s) we might need
e_shift=np.zeros(len(gnss_data.index))
n_shift=np.zeros(len(gnss_data.index))
v_shift=np.zeros(len(gnss_data.index))

In [ ]:
# let's make an offset correction!

# set the date of the offset (e.g. an earthquake)
off_date=eq_date
#off_date=np.datetime64('2013-05-01')


# we'll assume that we can difference the positions before and after the offset, to estimate the offset
e_off=(gnss_data.loc[gnss_data['Date']==(off_date+1), 'East (mm)'].values-
         gnss_data.loc[gnss_data['Date']==off_date, 'East (mm)'].values).item(0)

n_off=(gnss_data.loc[gnss_data['Date']==(off_date+1), 'North (mm)'].values-
         gnss_data.loc[gnss_data['Date']==off_date, 'North (mm)'].values).item(0)

v_off=(gnss_data.loc[gnss_data['Date']==(off_date+1), 'Vertical (mm)'].values-
         gnss_data.loc[gnss_data['Date']==off_date, 'Vertical (mm)'].values).item(0)

# index the dates after the offset
idx_postoff=gnss_data["Date"]>off_date

# and add the offsets to those post-offset dates 
e_shift[idx_postoff]=e_shift[idx_postoff]+e_off
n_shift[idx_postoff]=n_shift[idx_postoff]+n_off
v_shift[idx_postoff]=v_shift[idx_postoff]+v_off

In [ ]:
v_shift

In [ ]:
# as a check, let's remove the offsets from our detrended data

fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, figsize=(15,15))
fig.suptitle('{0:s} detrended and deoffsetted'.format(my_sta))

ax1.plot_date(gnss_data['Date'],gnss_data['East (mm)']-e_trend-e_shift,fmt='.')
ax1.set_ylabel('East (mm)')
ax1.xaxis.set_major_formatter(years_fmt)

ax2.plot_date(gnss_data['Date'],gnss_data['North (mm)']-n_trend-n_shift,fmt='.')
ax2.set_ylabel('North (mm)')
ax2.xaxis.set_major_formatter(years_fmt)

ax3.plot_date(gnss_data['Date'],gnss_data['Vertical (mm)']-v_trend-v_shift,fmt='.')
ax3.set_ylabel('Vertical (mm)')
ax3.xaxis.set_major_formatter(years_fmt)

fig.autofmt_xdate()

fig.show()

It may be that there are other offsets that we need to address? We can repeat the process (taking care not to reset the e_shift, n_shift and v_shift arrays), to try and account for any other offsets there might be. This might require a bit of trial and error, and/or checking station records and earthquake records to see if there is any potential cause for the offset...

In [ ]:
# look at the data either side of the trial offset date, a
off_date=np.datetime64('2012-12-01')
gnss_data.loc[((gnss_data['Date']>off_date-20)&(gnss_data['Date']<off_date+20))]

### 2.3 Postseismic transients

Postseismic transients are short(ish)-lived periods of accelerated deformation following large crustal earthquakes, caused by the stress changes induced by the fault slip in an earthquake. Mechanisms for this deformation include frictional afterslip (ongoing slip on parts of a fault or shear zone that did not slip fully in the earthquake) and viscoelastic relaxation (viscous flow in the ductile rocks of the lower crust and/or uppermost mantle). Often we infer a combination of these processes, and with different time-dependent behaviors at different depths.  

While a deep dive through all of the possible mechanisms and models may be overkill right now, we will attempt to fit a simple model to the postseismic displacements, $d_j$, in our time series:

$$d_j = A \ln\left[\frac{1+(t_j-t_0)}{\tau}\right] + d_0$$

where $A$ is a scaling constant, $\tau$ is a time constant, related to the speed of postseismic relaxation, $d_0$ is a static shift in displacement, and $t_j-t_0$ is time since the earthquake.

In [ ]:
# try a time constant, a priori
t_const=500

# set up somewhere to store our transient fits
e_trans=np.zeros(len(gnss_data.index))
n_trans=np.zeros(len(gnss_data.index))
v_trans=np.zeros(len(gnss_data.index))

# post-earthquake data index
idx_posteq=gnss_data["Date"]>eq_date

# extract some data for the postseismic period
t_posteq=gnss_data["Day_number"].loc[gnss_data["Date"]>eq_date].values \
          - gnss_data["Day_number"].loc[gnss_data["Date"]==eq_date].values

e_posteq=(gnss_data["East (mm)"].loc[gnss_data["Date"]>eq_date].values \
          - e_trend[gnss_data["Date"]>eq_date] - e_shift[gnss_data["Date"]>eq_date]).values
n_posteq=(gnss_data["North (mm)"].loc[gnss_data["Date"]>eq_date].values \
          - n_trend[gnss_data["Date"]>eq_date] - n_shift[gnss_data["Date"]>eq_date]).values
v_posteq=(gnss_data["Vertical (mm)"].loc[gnss_data["Date"]>eq_date].values \
          - v_trend[gnss_data["Date"]>eq_date] - v_shift[gnss_data["Date"]>eq_date]).values

# set up some design matrices
AT = np.vstack((np.log(1+t_posteq/t_const), np.ones(np.shape(t_posteq))))

# east component
Einv_e = np.diag(1/gnss_data['East Std. Deviation (mm)'].loc[gnss_data["Date"]>eq_date].values**2)
ATAinv_e = np.linalg.inv(np.matmul(AT,np.matmul(Einv_e,AT.T)))
ATd_e = np.matmul(AT,np.matmul(Einv_e,e_posteq))
m_e = np.matmul(ATAinv_e,ATd_e.T)  # amplitude and intercept of best-fit line
e_trans[idx_posteq]=e_trans[idx_posteq]+AT[0]*m_e[0]+AT[1]*m_e[1]

# north component
Einv_n = np.diag(1/gnss_data['North Std. Deviation (mm)'].loc[gnss_data["Date"]>eq_date].values**2)
ATAinv_n = np.linalg.inv(np.matmul(AT,np.matmul(Einv_n,AT.T)))
ATd_n = np.matmul(AT,np.matmul(Einv_n,n_posteq))
m_n = np.matmul(ATAinv_n,ATd_n.T)  # amplitude and intercept of best-fit line
n_trans[idx_posteq]=n_trans[idx_posteq]+AT[0]*m_n[0]+AT[1]*m_n[1]

# vertical component
Einv_v = np.diag(1/gnss_data['Vertical Std. Deviation (mm)'].loc[gnss_data["Date"]>eq_date].values**2)
ATAinv_v = np.linalg.inv(np.matmul(AT,np.matmul(Einv_v,AT.T)))
ATd_v = np.matmul(AT,np.matmul(Einv_v,v_posteq))
m_v = np.matmul(ATAinv_v,ATd_v.T)  # amplitude and intercept of best-fit line
v_trans[idx_posteq]=v_trans[idx_posteq]+AT[0]*m_v[0]+AT[1]*m_v[1]

In [ ]:
# as a check, let's remove the transients from our detrended, deoffsetted data

fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, figsize=(15,15))
fig.suptitle('{0:s} detrended, deoffsetted and detransiented'.format(my_sta))

ax1.plot_date(gnss_data['Date'],gnss_data['East (mm)']-e_trend-e_shift-e_trans,fmt='.')
ax1.set_ylabel('East (mm)')
ax1.xaxis.set_major_formatter(years_fmt)

ax2.plot_date(gnss_data['Date'],gnss_data['North (mm)']-n_trend-n_shift-n_trans,fmt='.')
ax2.set_ylabel('North (mm)')
ax2.xaxis.set_major_formatter(years_fmt)

ax3.plot_date(gnss_data['Date'],gnss_data['Vertical (mm)']-v_trend-v_shift-v_trans,fmt='.')
ax3.set_ylabel('Vertical (mm)')
ax3.xaxis.set_major_formatter(years_fmt)

fig.autofmt_xdate()

fig.show()

It seems that it is not possible to fit all of the postseismic transient on all of the components with a single time constant. This suggests that more than one mechanism, with more than one time constant, is in operation (afterslip and viscous relaxation, say). Not every component, however, is sensitive to all of the mechanisms $-$ for example, the vertical component is quite insensitive to afterslip, and can be acceptably fitted with a single, longer time constant.

And what is left in the vertical component, if we do that? It looks like a seasonal signal, mostly...

### 2.4 Seasonal signals

The vertical component of GNSS is particularly sensitive to changes in groundwater storage, which can impart a seasonal repeating signal to the time series. In some cases, particularly where stations are located near the edges of aquifers, this can affect one or more of the horizontal components too.

For a final decomposition step, we can try and remove this seasonal component from the time series. We can start with an annual component, and see what it looks like. The easiest way to model an annual component, with unknown phase and amplitude, is to simultaneously solve for the best sine and cosine waveform fits, assuming an annual recurrence, i.e.

$$d_i = A \sin\left(\frac{2\pi t_i}{365.25}\right) + B \cos\left(\frac{2\pi t_i}{365.25}\right) + d_0$$

...whose unknowns ($A$, $B$ and $d_0$) can be obtained with a straightforward least-squares inversion.

In [ ]:
# the vertical residual - whatever data are remaining after the other corrections
v_resid=gnss_data['Vertical (mm)'].values-v_trend-v_shift-v_trans

# set up a design matrix
AT = np.vstack((np.sin(2*np.pi*gnss_data["Day_number"]/365.25),np.cos(2*np.pi*gnss_data["Day_number"]/365.25),
                np.ones(np.shape(v_resid))))

# and invert
Einv_v = np.diag(1/gnss_data['Vertical Std. Deviation (mm)'].values**2)
ATAinv_v = np.linalg.inv(np.matmul(AT,np.matmul(Einv_v,AT.T)))
ATd_v = np.matmul(AT,np.matmul(Einv_v,v_resid))
m_v = np.matmul(ATAinv_v,ATd_v.T)  # sin/cosine amplitudes and y axis shift of best-fit line
v_seasonal = AT[0]*m_v[0]+AT[1]*m_v[1]+AT[2]*m_v[2]

In [ ]:
# and let's see what it looks like...
fig, ax = plt.subplots(1, figsize=(15,5))
fig.suptitle('{0:s} detrended, deoffsetted, detransiented and deseasonalized'.format(my_sta))

ax.plot_date(gnss_data['Date'],gnss_data['Vertical (mm)']-v_trend-v_shift-v_trans-v_seasonal,fmt='.')
ax.set_ylabel('Vertical (mm)')
ax.xaxis.set_major_formatter(years_fmt)

fig.autofmt_xdate()

fig.show()